# BERTopic Exploration of the Stimmmungs- und Lageberichte Database Files
**Author:** Christopher Thomas Goodwin

**Creation Date:** 2024.04.10

**Summary:** Uses BERTopic modelling to explore the data of the NSHWE Stimmungs- und Lageberichte files

In [ ]:
from bertopic import BERTopic
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
with open("/Users/cgoodwin/Documents/Programming/Python/NSHWEDatabaseMining/data/stimmungs_data_sentences.json", "r", encoding="utf-8") as f:
    files = json.load(f)
    # files loaded in as dictionary with strings of 0... length of files
    
    # we want just the textual data, the report from each entry
    reports = []
    for i in range(len(files)):
        reports.append(files[str(i)]["report"]) # iterate through dictionary and append report

In [ ]:
# set up vectorizer for German stopwords
german_stop_words = stopwords.words('german')
vectorizer_model = CountVectorizer(stop_words=german_stop_words)

# use multilingual model and apply German stopwords vectorizer model
topic_model = BERTopic(embedding_model="paraphrase-multilingual-MiniLM-L12-v2", vectorizer_model=vectorizer_model, language="multilingual", nr_topics=50)

In [ ]:
topics, probs = topic_model.fit_transform(reports)

In [ ]:
topic_model.get_topic_info()

In [ ]:
topic_model.get_topic(44)